In [1]:
from dotenv import load_dotenv

_ = load_dotenv()

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
!pip install langgraph-checkpoint-sqlite


In [4]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

In [5]:
import requests
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from typing import List, Optional
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
import json 
class UserProfile(BaseModel):
    name: Optional[str]
    age: Optional[int]
    gender: Optional[str]
    occupation: Optional[str]

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]
    user_profile: UserProfile

ONBOARDING_USER_CREATE_PROMPT = """You are the Big Boss bot: a helpful assistant that recommends movies to users.
 In the first step, you will ask user information to create a user profile. The information you need to ask is:
 - name
 - age
 - gender
 - occupation

Currently the user profile that you have is:
{user_profile}

Ask the user for the information that you don't have. 
"""


USER_CREATE_STATE_PROMPT = """You are the Big Boss bot: a helpful assistant that recommends movies to users.
You already have a user profile with the following information:
{user_profile}

Now you will thanks the user for the information and ask for the next information.
"""

EXTRACTION_USER_CREATE_PROMPT = """You are an extractor of user profile from a text.
 The text is a conversation between a user and a bot.
 The user profile is a dictionary with the following keys:
 - name
 - age
 - gender
 - occupation
You will receive a conversation message and you will extract the user profile from the message. Extract the user profile to a JSON object. 
Example:
```
User: Hello, my name is John Doe. I am 30 years old. I am a software engineer.
Bot: Hello, John. Nice to meet you.
```
The user profile is:
```json
{
    "name": "John Doe",
    "age": 30,
    "gender": "male",
    "occupation": "software engineer"
}
"""


    

In [6]:
import requests
import os

class Agent:
    def __init__(self, checkpointer):
        self.rec_url = os.getenv("RECOMMENDATION_API_URL")
        graph = StateGraph(AgentState)
        graph.add_node("user_profile_prompt", self.user_profile_prompt)
        graph.add_node("user_profile_extraction", self.user_profile_extraction)
        graph.add_node("create_user_profile", self.create_user_profile)

        graph.add_edge("user_profile_prompt", "user_profile_extraction")
        graph.add_conditional_edges("user_profile_extraction", self.exists_user_profile, {True: END, False: "user_profile_prompt"})
        graph.add_edge("user_profile_extraction", "create_user_profile")
        graph.add_edge("create_user_profile", END)


        graph.set_entry_point("user_profile_prompt")

        self.graph = graph.compile(checkpointer=checkpointer)
        self.model = ChatOpenAI(model="gpt-5-nano", temperature=0)

    def exists_user_profile(self, state: AgentState) -> bool:
        user_profile = state.get("user_profile")
        if user_profile is None:
            return False
        return user_profile.age is not None and user_profile.gender is not None and user_profile.occupation is not None

    def user_profile_prompt(self, state: AgentState) -> AgentState:
        print("in user_profile_prompt")
        profile = state.get("user_profile")
        profile_json = json.dumps(profile.model_dump() if profile else {}, ensure_ascii=False)
        prompt = ChatPromptTemplate.from_template(ONBOARDING_USER_CREATE_PROMPT)


        rendered = prompt.format(user_profile=profile_json, user_input=state['messages'][-1].content)
        messages = [rendered] + state['messages']
        model_message = self.model.invoke(messages)
        return {'messages': [model_message]}

    def create_user_profile(self, state: AgentState) -> AgentState:
        print("in create_user_profile")
        url = self.rec_url.rstrip("/") + "/users/"
        profile = state.get("user_profile")
        if not profile:
            raise ValueError("User profile is not found")
        name = profile.name
        age = profile.age
        gender = profile.gender
        occupation = profile.occupation
        payload = {
            "name": name,
            "age": age,
            "gender": gender,
            "occupation": occupation,
        }
        r = requests.post(url, json=payload, timeout=30)
        return { "messages": [r.json()], "user_profile": profile}


    def user_profile_extraction(self, state: AgentState) -> AgentState:
        print("in user_profile_extraction")
        parser = JsonOutputParser()
        messages = [SystemMessage(content=EXTRACTION_USER_CREATE_PROMPT)] + state["messages"]
        model_message = self.model.invoke(messages)
        content = model_message.content
        data = parser.parse(content)

        profile = UserProfile(
            name=data.get("name"),
            age=int(data.get("age")) if data.get("age") else None,
            gender=data.get("gender"),
            occupation=data.get("occupation"),
        )


        return {"messages": [model_message], "user_profile": profile} 

In [7]:
agent=Agent(checkpointer=memory)

In [8]:
from langchain_core.messages import HumanMessage, AIMessage

messages = [
    HumanMessage(content="Hello. 30 years, male"),
    AIMessage(content="Please tell your occupation"),
    HumanMessage(content="I am a software engineer")
]

thread = {"configurable": {"thread_id": "1"}}
for event in agent.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

in user_profile_prompt
[AIMessage(content='Nice to meet you! I have your details as: age 30, gender male, occupation software engineer. What is your name so I can complete your profile? If you prefer, you can share a preferred name or nickname.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 439, 'prompt_tokens': 112, 'total_tokens': 551, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 384, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CGb84Ra3uoM0TVdDgZRsfQmY1AigE', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--b8151a0b-9d4b-4a8b-814a-3da7b31b62af-0', usage_metadata={'input_tokens': 112, 'output_tokens': 439, 'total_tokens': 551, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 

In [9]:
from fastapi import FastAPI
from langgraph.server import create_server
import nest_asyncio, uvicorn

# reuse your existing memory/Agent
app = create_server(agent.graph)  # agent.graph is your compiled graph

nest_asyncio.apply()
uvicorn.run(app, host="127.0.0.1", port=8123)

ModuleNotFoundError: No module named 'fastapi'